In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import h5py
import numpy as np
import tensorflow as tf
import sklearn.metrics as metrics
import itertools
import matplotlib.pylab as plt

from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
from itertools import cycle
from scipy import interp
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size

In [ ]:
%cd ... # <--- INSERT HERE (...) THE PATH OF THE FOLDER

In [ ]:
ds_train_h5f = h5py.File("./train-zyx-250x190x270.h5",'r')
ds_val_h5f = h5py.File("./val-zyx-250x190x270.h5",'r')
ds_test_h5f = h5py.File("./test-zyx-250x190x270.h5",'r')

X_train = ds_train_h5f["train_X"]
Y_train = ds_train_h5f["train_Y"]

X_val = ds_val_h5f["val_X"]
Y_val = ds_val_h5f["val_Y"]

X_test = ds_test_h5f["test_X"]
Y_test = ds_test_h5f["test_Y"]

#print(X_train.shape)
#print(Y_train.shape)

#print(X_val.shape)
#print(Y_val.shape)

#print(X_test.shape)
#print(Y_test.shape)

idx_map_train = np.arange(X_train.shape[0])
np.random.shuffle(idx_map_train)

idx_map_val = np.arange(X_val.shape[0])
np.random.shuffle(idx_map_val)

idx_map_test = np.arange(X_test.shape[0])
np.random.shuffle(idx_map_test)

In [ ]:
def generate_batches_from_train_hdf5_file(hdf5_file, batch_size, idx_map):
  file_size = len(hdf5_file['train_Y'])

  while 1:
    # Count how many entries we have read
    n_entries = 0
    # As long as we haven't read all entries from the file: keep reading
    while n_entries < (file_size - batch_size):
      # Start the next batch at index 0
      # Create numpy arrays of input data (features)
      xs = hdf5_file['train_X'][n_entries: n_entries + batch_size,:,:,:]
      xs = np.array(xs)
      xs = xs[..., np.newaxis]

      ys = hdf5_file['train_Y'][n_entries:n_entries + batch_size]
      ys = np.array(ys)

      # We have read one more batch from this file
      n_entries += batch_size
      yield (xs, ys)

In [ ]:
def generate_batches_from_val_hdf5_file(hdf5_file, batch_size, idx_map):
  file_size = len(hdf5_file['val_Y'])

  while 1:
    # Count how many entries we have read
    n_entries = 0
    # As long as we haven't read all entries from the file: keep reading
    while n_entries < (file_size - batch_size):
      # Start the next batch at index 0
      # Create numpy arrays of input data (features)
      xs = hdf5_file['val_X'][n_entries: n_entries + batch_size,:,:,:]
      xs = np.array(xs)
      xs = xs[..., np.newaxis]

      ys = hdf5_file['val_Y'][n_entries:n_entries + batch_size]
      ys = np.array(ys)

      # We have read one more batch from this file
      n_entries += batch_size
      yield (xs, ys)

In [ ]:
def generate_batches_from_test_hdf5_file(hdf5_file, batch_size, idx_map):
  file_size = len(hdf5_file['test_Y'])

  while 1:
    # Count how many entries we have read
    n_entries = 0
    # As long as we haven't read all entries from the file: keep reading
    while n_entries < (file_size - batch_size):
      # Start the next batch at index 0
      # Create numpy arrays of input data (features)
      xs = hdf5_file['test_X'][n_entries: n_entries + batch_size,:,:,:]
      xs = np.array(xs)
      xs = xs[..., np.newaxis]

      ys = hdf5_file['test_Y'][n_entries:n_entries + batch_size]
      ys = np.array(ys)

      # We have read one more batch from this file
      n_entries += batch_size
      yield (xs, ys)

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
  plt.figure(figsize=(6, 6), dpi=80)

  im = plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)

  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, round(cm[i, j],2),
      horizontalalignment="center",
      color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

  ax = plt.gca()
  divider = make_axes_locatable(ax)
  cax = divider.append_axes("right", size="5%", pad=0.05)
  plt.colorbar(im, cax=cax)

In [ ]:
def getVGG16Model(verbose=True):
  """
  Builds the VGG16 2D CNN (non-pretrained on ImageNet -> weights=None)

    Parameters
    ----------
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    vgg - Sequential
          The instantiated model
  """
  vgg = VGG16(include_top=False, weights=None, input_shape=(190,270,1)); # this way the input_shape can be modified (default is (224,224,3)) and the training is from scratch
  if verbose:
    vgg.summary()

  for layer in vgg.layers:
     layer.trainable = True;
  #for layer in vgg.layers:
  #  layer.trainable = False;

  return vgg

In [ ]:
def getVGG19Model(verbose=True):
  """
  Builds the VGG19 2D CNN (non-pretrained on ImageNet -> weights=None)

    Parameters
    ----------
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    vgg - Sequential
          The instantiated model
  """
  vgg = VGG19(include_top=False, weights=None, input_shape=(190,270,1));
  if verbose:
    vgg.summary()

  for layer in vgg.layers:
    layer.trainable = True;
  #for layer in vgg.layers:
  #  layer.trainable = False;

  return vgg

In [ ]:
def getResNet50V2Model(verbose=True):
  """
  Builds the ResNet50V2 2D CNN (non-pretrained on ImageNet -> weights=None)

    Parameters
    ----------
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    resnet - Sequential
              The instantiated model
  """
  resnet = ResNet50V2(include_top=False, weights=None, input_shape=(190,270,1));
  if verbose:
    resnet.summary()

  for layer in resnet.layers:
    layer.trainable = True;
  #for layer in vgg.layers:
  #  layer.trainable = False;

  return resnet

In [ ]:
def getXceptionModel(verbose=True):
  """
  Builds the Xception 2D CNN (non-pretrained on ImageNet -> weights=None)

    Parameters
    ----------
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    xcep - Sequential
           The instantiated model
  """
  xcep = Xception(include_top=False, weights=None, input_shape=(190,270,1));
  if verbose:
    xcep.summary()

  for layer in xcep.layers:
    layer.trainable = True;
  #for layer in vgg.layers:
  #  layer.trainable = False;

  return xcep

In [ ]:
def getNasnetModel(verbose=True):
  """
  Builds the NASNet Mobile 2D CNN (non-pretrained on ImageNet -> weights=None)

    Parameters
    ----------
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    nasn - Sequential
           The instantiated model
  """
  nasn = NASNetMobile(include_top=False, weights=None, input_shape=(190,270,1));
  if verbose:
    nasn.summary()

  for layer in nasn.layers:
    layer.trainable = True;
  #for layer in vgg.layers:
  #  layer.trainable = False;

  return nasn

In [ ]:
def getInceptionResNetV2Model(verbose=True):
  """
  Builds the Inception-ResNetV2 2D CNN (non-pretrained on ImageNet -> weights=None)

    Parameters
    ----------
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    incres - Sequential
              The instantiated model
  """
  incres = InceptionResNetV2(include_top=False, weights=None, input_shape=(190,270,1));
  if verbose:
    incres.summary()

  for layer in incres.layers:
    layer.trainable = True;
  #for layer in vgg.layers:
  #  layer.trainable = False;

  return incres

In [ ]:
### 1 ###

input_shape1 = (190,270,1)

def getConvModel(verbose=True):

  model = models.Sequential()

  model.add(layers.InputLayer(input_shape=input_shape1))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_1"))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_2"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_1"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_3"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_4"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_2"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_5"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_6"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_7"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_8"))
  model.add(layers.BatchNormalization(axis=-1, # if channel last (axis=1 if channel first)
                                      momentum=0.99,
                                      epsilon=0.001,
                                      center=True,
                                      scale=True,
                                      beta_initializer="zeros",
                                      gamma_initializer="ones",
                                      moving_mean_initializer="zeros",
                                      moving_variance_initializer="ones",
                                      name="batch_norm_1"))
  model.add(layers.Activation("relu",
                              name="activation_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_3"))

  if verbose:
    model.summary()

  return model

In [ ]:
### 2 ###

input_shape1 = (190,270,1)

def getConvModel(verbose=True):

  model = models.Sequential()

  model.add(layers.InputLayer(input_shape=input_shape1))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_1"))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_2"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_1"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_3"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_4"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_5"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_2"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_6"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_7"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_8"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_9"))
  model.add(layers.BatchNormalization(axis=-1,
                                      momentum=0.99,
                                      epsilon=0.001,
                                      center=True,
                                      scale=True,
                                      beta_initializer="zeros",
                                      gamma_initializer="ones",
                                      moving_mean_initializer="zeros",
                                      moving_variance_initializer="ones",
                                      name="batch_norm_1"))
  model.add(layers.Activation("relu",
                              name="activation_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_3"))

  if verbose:
    model.summary()

  return model

In [ ]:
### 3 ###

input_shape1 = (190,270,1)

def getConvModel(verbose=True):

  model = models.Sequential()

  model.add(layers.InputLayer(input_shape=input_shape1))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_1"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_2"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_3"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_2"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_4"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_5"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_6"))
  model.add(layers.BatchNormalization(axis=-1,
                                      momentum=0.99,
                                      epsilon=0.001,
                                      center=True,
                                      scale=True,
                                      beta_initializer="zeros",
                                      gamma_initializer="ones",
                                      moving_mean_initializer="zeros",
                                      moving_variance_initializer="ones",
                                      name="batch_norm_1"))
  model.add(layers.Activation("relu",
                              name="activation_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_3"))

  if verbose:
    model.summary()

  return model

In [ ]:
### 4 ###

input_shape1 = (190,270,1)

def getConvModel(verbose=True):

  model = models.Sequential()

  model.add(layers.InputLayer(input_shape=input_shape1))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_1"))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_2"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_1"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_3"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_4"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_2"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_5"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_6"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_7"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_8"))
  model.add(layers.BatchNormalization(axis=-1,
                                      momentum=0.99,
                                      epsilon=0.001,
                                      center=True,
                                      scale=True,
                                      beta_initializer="zeros",
                                      gamma_initializer="ones",
                                      moving_mean_initializer="zeros",
                                      moving_variance_initializer="ones",
                                      name="batch_norm_1"))
  model.add(layers.Activation("relu",
                              name="activation_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_3"))

  if verbose:
    model.summary()

  return model

In [ ]:
### 5 ###

input_shape1 = (190,270,1)

def getConvModel(verbose=True):

  model = models.Sequential()

  model.add(layers.InputLayer(input_shape=input_shape1))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_1"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_2"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_2"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_3"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_3"))
  model.add(layers.Conv2D(filters=256,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_4"))
  model.add(layers.Conv2D(filters=256,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_5"))
  model.add(layers.Conv2D(filters=256,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_6"))
  model.add(layers.BatchNormalization(axis=-1,
                                      momentum=0.99,
                                      epsilon=0.001,
                                      center=True,
                                      scale=True,
                                      beta_initializer="zeros",
                                      gamma_initializer="ones",
                                      moving_mean_initializer="zeros",
                                      moving_variance_initializer="ones",
                                      name="batch_norm_1"))
  model.add(layers.Activation("relu",
                              name="activation_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_4"))

  if verbose:
    model.summary()

  return model

In [ ]:
### 6 ###

input_shape1 = (190,270,1)

def getConvModel(verbose=True):

  model = models.Sequential()

  model.add(layers.InputLayer(input_shape=input_shape1))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_1"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_2"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_2"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_3"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_3"))
  model.add(layers.Conv2D(filters=256,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_4"))
  model.add(layers.Conv2D(filters=256,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_5"))
  model.add(layers.BatchNormalization(axis=-1,
                                      momentum=0.99,
                                      epsilon=0.001,
                                      center=True,
                                      scale=True,
                                      beta_initializer="zeros",
                                      gamma_initializer="ones",
                                      moving_mean_initializer="zeros",
                                      moving_variance_initializer="ones",
                                      name="batch_norm_1"))
  model.add(layers.Activation("relu",
                              name="activation_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_4"))
  model.add(layers.Conv2D(filters=512,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_6"))
  model.add(layers.Conv2D(filters=512,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_7"))
  model.add(layers.BatchNormalization(axis=-1,
                                      momentum=0.99,
                                      epsilon=0.001,
                                      center=True,
                                      scale=True,
                                      beta_initializer="zeros",
                                      gamma_initializer="ones",
                                      moving_mean_initializer="zeros",
                                      moving_variance_initializer="ones",
                                      name="batch_norm_2"))
  model.add(layers.Activation("relu",
                              name="activation_2"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_5"))

  if verbose:
    model.summary()

  return model

In [ ]:
### 7 ###

input_shape1 = (190,270,1)

def getConvModel(verbose=True):

  model = models.Sequential()

  model.add(layers.InputLayer(input_shape=input_shape1))
  model.add(layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_1"))
  model.add(layers.Conv2D(filters=64,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_2"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_2"))
  model.add(layers.Conv2D(filters=128,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_3"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_3"))
  model.add(layers.Conv2D(filters=256,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_4"))
  model.add(layers.Conv2D(filters=256,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_5"))
  model.add(layers.Conv2D(filters=256,
                          kernel_size=(3,3),
                          padding="same",
                          activation="relu",
                          name="conv2d_6"))
  model.add(layers.BatchNormalization(axis=-1,
                                      momentum=0.99,
                                      epsilon=0.001,
                                      center=True,
                                      scale=True,
                                      beta_initializer="zeros",
                                      gamma_initializer="ones",
                                      moving_mean_initializer="zeros",
                                      moving_variance_initializer="ones",
                                      name="batch_norm_1"))
  model.add(layers.Activation("relu",
                              name="activation_1"))
  model.add(layers.MaxPool2D(pool_size=(2,2),
                             strides=(2,2),
                             name="max_pool2d_4"))

  if verbose:
    model.summary()

  return model

In [ ]:
### 1 ###

input_shape2 = (250,190,270,1)

def getNeuralNet(getConvModel, verbose=True):
  """
  Creates the model composed of time-distributed 2D CNN + ConvLSTM with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel - Callable[[bool], Sequential]
                   Function that instantiates the Convolutional model to be applied in a time distributed fashion
                   The parameter is a boolean True for verbose output, False otherwise
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    model - Sequential
            The instantiated model
  """
  model = models.Sequential()
  model.add(layers.TimeDistributed(getConvModel(),
                                   input_shape=input_shape2,
                                   name="time_distributed_1"))
  model.add(layers.ConvLSTM2D(128,
                              kernel_size=(3,3),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_2"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_2"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
### 2 ###

input_shape2 = (250,190,270,1)

def getNeuralNet(getConvModel, verbose=True):
  """
  Creates the model composed of time-distributed 2D CNN + ConvLSTM with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel - Callable[[bool], Sequential]
                   Function that instantiates the Convolutional model to be applied in a time distributed fashion
                   The parameter is a boolean True for verbose output, False otherwise
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    model - Sequential
            The instantiated model
  """
  model = models.Sequential()
  model.add(layers.TimeDistributed(getConvModel(),
                                   input_shape=input_shape2,
                                   name="time_distributed_1"))
  model.add(layers.ConvLSTM2D(128,
                              kernel_size=(3,3),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(256,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_2"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_2"))
  model.add(layers.Dropout(0.5,
                           name="dropout_3"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_3"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
### 3 ###

input_shape2 = (250,190,270,1)

def getNeuralNet(getConvModel, verbose=True):
  """
  Creates the model composed of time-distributed 2D CNN + ConvLSTM with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel - Callable[[bool], Sequential]
                   Function that instantiates the Convolutional model to be applied in a time distributed fashion
                   The parameter is a boolean True for verbose output, False otherwise
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    model - Sequential
            The instantiated model
  """
  model = models.Sequential()
  model.add(layers.TimeDistributed(getConvModel(),
                                   input_shape=input_shape2,
                                   name="time_distributed_1"))
  model.add(layers.ConvLSTM2D(256,
                              kernel_size=(3,3),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(256,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_2"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_2"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
### 4 ###

input_shape2 = (250,190,270,1)

def getNeuralNet(getConvModel, verbose=True):
  """
  Creates the model composed of time-distributed 2D CNN + ConvLSTM with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel - Callable[[bool], Sequential]
                   Function that instantiates the Convolutional model to be applied in a time distributed fashion
                   The parameter is a boolean True for verbose output, False otherwise
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    model - Sequential
            The instantiated model
  """
  model = models.Sequential()
  model.add(layers.TimeDistributed(getConvModel(),
                                   input_shape=input_shape2,
                                   name="time_distributed_1"))
  model.add(layers.ConvLSTM2D(128,
                              kernel_size=(3,3),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.7,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.3,
                           name="dropout_2"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_2"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
### 5 ###

input_shape2 = (250,190,270,1)

def getNeuralNet(getConvModel, verbose=True):
  """
  Creates the model composed of time-distributed 2D CNN + ConvLSTM with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel - Callable[[bool], Sequential]
                   Function that instantiates the Convolutional model to be applied in a time distributed fashion
                   The parameter is a boolean True for verbose output, False otherwise
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    model - Sequential
            The instantiated model
  """
  model = models.Sequential()
  model.add(layers.TimeDistributed(getConvModel(),
                                   input_shape=input_shape2,
                                   name="time_distributed_1"))
  model.add(layers.ConvLSTM2D(64,
                              kernel_size=(3,3),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.6,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_2"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_2"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
### 6 ###

input_shape2 = (250,190,270,1)

def getNeuralNet(getConvModel, verbose=True):
  """
  Creates the model composed of time-distributed 2D CNN + ConvLSTM with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel - Callable[[bool], Sequential]
                   Function that instantiates the Convolutional model to be applied in a time distributed fashion
                   The parameter is a boolean True for verbose output, False otherwise
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    model - Sequential
            The instantiated model
  """
  model = models.Sequential()
  model.add(layers.TimeDistributed(getConvModel(),
                                   input_shape=input_shape2,
                                   name="time_distributed_1"))
  model.add(layers.ConvLSTM2D(64,
                              kernel_size=(3,3),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_2"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_2"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
### 7 ###

input_shape2 = (250,190,270,1)

def getNeuralNet(getConvModel, verbose=True):
  """
  Creates the model composed of time-distributed 2D CNN + ConvLSTM with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel - Callable[[bool], Sequential]
                   Function that instantiates the Convolutional model to be applied in a time distributed fashion
                   The parameter is a boolean True for verbose output, False otherwise
    verbose - bool
              If True prints the model summary (default True)

    Returns
    ----------
    model - Sequential
            The instantiated model
  """
  model = models.Sequential()
  model.add(layers.TimeDistributed(getConvModel(),
                                   input_shape=input_shape2,
                                   name="time_distributed_1"))
  model.add(layers.ConvLSTM2D(64,
                              kernel_size=(3,3),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.6,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.2,
                           name="dropout_2"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_2"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
#model = getNeuralNet(getVGG16Model)
#model = getNeuralNet(getVGG19Model, True)
#model = getNeuralNet(getResNet50V2Model, True)
#model = getNeuralNet(getXceptionModel, True)
#model = getNeuralNet(getNasnetModel, True)
#model = getNeuralNet(getInceptionResNetV2Model, True)
model = getNeuralNet(getConvModel, True)

In [ ]:
my_batch_size = 1
my_steps_per_epoch = int(720//my_batch_size)
my_validation_steps = int(50//my_batch_size)
my_epochs = 50
my_callbacks = [
                tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1, restore_best_weights=True),
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-vgg16+convlstm.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-vgg19+convlstm.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-resnet+convlstm.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-xcep+convlstm.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-nasn+convlstm.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-incres+convlstm.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-tdcnn+convlstm-1.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-tdcnn+convlstm-2.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-tdcnn+convlstm-3.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-tdcnn+convlstm-4.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-tdcnn+convlstm-5.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-tdcnn+convlstm-6.h5", save_best_only=True, mode='min', monitor='val_loss')
                #tf.keras.callbacks.ModelCheckpoint(filepath="./model-tdcnn+convlstm-7.h5", save_best_only=True, mode='min', monitor='val_loss')
]

generator_train = generate_batches_from_train_hdf5_file(ds_train_h5f, 1, idx_map_train)
generator_val = generate_batches_from_val_hdf5_file(ds_val_h5f, 1, idx_map_val)

#model.layers[0].trainable = False # to exclude CNN training

history = model.fit(generator_train, validation_data=generator_val,
                    steps_per_epoch=my_steps_per_epoch, validation_steps=my_validation_steps, epochs=my_epochs,
                    callbacks=my_callbacks)

In [ ]:
### 1 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-tdcnn+convlstm-1.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-tdcnn+convlstm-1.csv',
#           probs,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-tdcnn+convlstm-1.csv',
#           Y_test,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('1 - specificity', fontsize=18)
plt.ylabel('sensitivity', fontsize=18)
plt.title('ROC curve', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
### 2 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-tdcnn+convlstm-2.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-tdcnn+convlstm-2.csv',
#           probs,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-tdcnn+convlstm-2.csv',
#           Y_test,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('1 - specificity', fontsize=18)
plt.ylabel('sensitivity', fontsize=18)
plt.title('ROC curve', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
### 3 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-tdcnn+convlstm-3.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-tdcnn+convlstm-3.csv',
#           probs,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-tdcnn+convlstm-3.csv',
#           Y_test,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('1 - specificity', fontsize=18)
plt.ylabel('sensitivity', fontsize=18)
plt.title('ROC curve', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
### 4 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-tdcnn+convlstm-4.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-tdcnn+convlstm-4.csv',
#           probs,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-tdcnn+convlstm-4.csv',
#           Y_test,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('1 - specificity', fontsize=18)
plt.ylabel('sensitivity', fontsize=18)
plt.title('ROC curve', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
### 5 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-tdcnn+convlstm-5.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-tdcnn+convlstm-5.csv',
#           probs,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-tdcnn+convlstm-5.csv',
#           Y_test,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('FP rate', fontsize=18)
plt.ylabel('TP rate', fontsize=18)
plt.title('ROC curve', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
### 6 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-tdcnn+convlstm-6.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-tdcnn+convlstm-6.csv',
#           probs,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-tdcnn+convlstm-6.csv',
#           Y_test,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('1 - specificity', fontsize=18)
plt.ylabel('sensitivity', fontsize=18)
plt.title('ROC curve', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
### 7 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-tdcnn+convlstm-7.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-tdcnn+convlstm-7.csv',
#           probs,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-tdcnn+convlstm-7.csv',
#           Y_test,
#           delimiter=',',
#           fmt='%1.3f',
#           header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('1 - specificity', fontsize=18)
plt.ylabel('sensitivity', fontsize=18)
plt.title('ROC curve', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()